# pls work

In [27]:
!pip install librosa pandas
!pip install torch==2.6.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 5.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.1/150.1 MB 5.8 MB/s eta 0:00:00:00:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.2/253.2 MB 3.9 MB/s eta 0:00:00:00:0100:01m
  Attempting uninstall: triton
    Found existing installation: triton 3.1.0
    Uninsta

In [28]:
import pandas as pd
import librosa
import numpy as np
import torch
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader

In [29]:
import pandas as pd

# import the csv file
# df_labelconvertion = pd.read_csv('fma_metadata/tracks.csv', index_col=1)
df_labelconvertion = pd.read_csv('/kaggle/input/fma-free-music-archive-small-medium/fma_metadata/tracks.csv', index_col=1) #with kaggle

# rename columns to actual names
df_labelconvertion.rename(columns={'Unnamed: 0': 'track_id', 'track.7': 'genre_top'}, inplace=True)

# make df of only the important columns
df_labelconvertion = df_labelconvertion[['track_id', 'genre_top']]

# set all NA values to other 
df_labelconvertion['genre_top'] = df_labelconvertion['genre_top'].fillna('Other')

# reset index from albums to normal index
df_labelconvertion = df_labelconvertion.reset_index(drop=True)

# make all values in the track_id column strings
df_labelconvertion['track_id'] = df_labelconvertion['track_id'].astype(str)

# remove first two rows
df_labelconvertion = df_labelconvertion.iloc[2:]


df_labelconvertion


/tmp/ipykernel_31/2803144048.py:5: DtypeWarning: Columns (0,1,5,6,8,12,18,20,21,22,24,33,34,38,39,44,47,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df_labelconvertion = pd.read_csv('/kaggle/input/fma-free-music-archive-small-medium/fma_metadata/tracks.csv', index_col=1) #with kaggle


,track_id,genre_top
2,2,Hip-Hop
3,3,Hip-Hop
4,5,Hip-Hop
5,10,Pop
6,20,Other
...,...,...
106571,155316,Rock
106572,155317,Rock
106573,155318,Rock
106574,155319,Rock


In [30]:
# print unique values in the genre_top column
unique_genres = df_labelconvertion['genre_top'].unique()
print(f"Unique genres: {unique_genres}")

Unique genres: ['Hip-Hop' 'Pop' 'Other' 'Rock' 'Experimental' 'Folk' 'Jazz' 'Electronic'
 'Spoken' 'International' 'Soul-RnB' 'Blues' 'Country' 'Classical'
 'Old-Time / Historic' 'Instrumental' 'Easy Listening']


In [31]:
import os

df_labelnumber = pd.DataFrame(columns=['labelnumber', 'unique genres'])
labelnumber = {}
for i, genre in enumerate(unique_genres):
    labelnumber[genre] = i
    
df_labelnumber = pd.DataFrame(labelnumber, index=[0])
# make rows into columns
df_labelnumber = df_labelnumber.transpose()

# save to csv file
# df_labelnumber.to_csv('fma_metadata/labelnumber.csv', index= True)

###kaggle###
os.makedirs("fma_metadata", exist_ok=True)
df_labelnumber.to_csv('/kaggle/working/fma_metadata/labelnumber.csv', index= True)
###

In [32]:
df_labelnumbers = pd.read_csv('fma_metadata/labelnumber.csv')

df_labelnumbers.rename(columns={'Unnamed: 0': 'genre', '0': 'number'}, inplace=True)

df_labelnumbers

,genre,number
0,Hip-Hop,0
1,Pop,1
2,Other,2
3,Rock,3
4,Experimental,4
5,Folk,5
6,Jazz,6
7,Electronic,7
8,Spoken,8
9,International,9


In [33]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import os
import torch

class SpectrogramDataset(Dataset):
    def __init__(self, file_paths, sr=44100, n_fft=1024, hop_length=512, window='hann', transform=None):
        self.sr = sr
        self.n_fft = n_fft
        self.hop_length = hop_length
        self.window = window
        self.transform = transform
        self.file_paths = file_paths
                    
    def __len__(self):
        return len(self.file_paths)
    
    def __getitem__(self, idx):
        file_path = self.file_paths[idx]
        y, sr = librosa.load(file_path, sr=self.sr, mono = True)
        y = y[:10 * self.sr]
        D = librosa.stft(y, n_fft=self.n_fft, hop_length=self.hop_length, window=self.window)
        D_real = np.real(D)
        D_imag = np.imag(D)
        D_real = torch.tensor(D_real, dtype=torch.float32)
        D_imag = torch.tensor(D_imag, dtype=torch.float32)
        
        # concat real and imaginary parts to 2 channels
        D = torch.stack((D_real, D_imag), dim=0)
        # get the label from the file name and convert to genre
        track_id = os.path.basename(file_path).replace('.mp3', '').lstrip('0')
        # try to get the label from the dataframe
        try:
            # label = df_labelconvertion.query('track_id == @track_id')['genre_top'].values[0]
            label = df_labelconvertion.query('track_id == @track_id')['genre_top'].values[0]
            label = torch.tensor(df_labelnumbers.query('genre == @label')['number'].values[0])
        except IndexError:
            # if not found, set label to 'Other'
            print(f"Track ID: {track_id} not found in dataframe")
            label = 'Other'
        
        return D, label

In [34]:
# create dataloader
batch_size = 32
num_workers = 4
# file_directory = 'fma_small/fma_small/'
file_directory = '/kaggle/input/fma-free-music-archive-small-medium/fma_small/fma_small' # kaggle
# train test split
from sklearn.model_selection import train_test_split
# get all file paths
file_paths = []
for root, _, files in os.walk(file_directory):
    for file in files:
        if file.endswith('.mp3'):
            file_paths.append(os.path.join(root, file))
            
# split the file paths into train and test
train_paths, test_paths = train_test_split(file_paths, test_size=0.2, random_state=42)


train_dataset = SpectrogramDataset(train_paths)
test_dataset = SpectrogramDataset(test_paths)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [35]:
# check the shape of the data
for batch in train_dataloader:
    print(batch[0].shape)
    print(batch[1].shape)
    break

torch.Size([32, 2, 513, 862])
torch.Size([32])


In [36]:
print(torch.__version__)

2.5.1+cu124


In [37]:

# import cvae_actual_i_think

### kaggle ###
import sys
sys.path.append('/kaggle/input/cvae-actual-i-think')
###

import cvae_actual_i_think

import importlib
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import CSVLogger

importlib.reload(cvae_actual_i_think) # Reload the cvae module to ensure it's up to date

model = cvae_actual_i_think.ConvCVAEPL()

# print(hasattr(model, 'forward'))

logger = CSVLogger("logs", name="train_10_epochs")

# Initialize the PyTorch Lightning Trainer
trainer = Trainer(max_epochs=5, accelerator="auto", logger=logger) #TODO


# Train the model
import time

start_time = time.time()
print("Training started")

trainer.fit(model, train_dataloader, test_dataloader)
print("Training finished")
end_time = time.time()
print(f"Training time: {end_time - start_time} seconds")

Training started


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


RuntimeError: output padding must be smaller than either stride or dilation, but got output_padding_height: 1 output_padding_width: 2 stride_height: 2 stride_width: 2 dilation_height: 1 dilation_width: 1